In [1]:
import cv2
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from centroid_tracker import CentroidTracker

In [2]:
model = YOLO("yolov8n-face.pt")

centroid_tracker = CentroidTracker()
deep_sort_tracker = DeepSort(max_age=30)

track_history = {}
tracker_mode = "deepsort"

D:\Studies\venvs\cvprojenv\Lib\site-packages\deep_sort_realtime\embedder\embedder_pytorch.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [6]:
cap = cv2.VideoCapture(0)

print("Press '1' for Centroid Tracker, '2' for Deep SORT, 'q' or ESC to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)[0]
    rects = []

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        conf = float(box.conf[0].cpu())
        if conf > 0.5:
            rects.append((x1, y1, x2, y2))

    # Centroid Tracker
    if tracker_mode == "centroid":
        objects = centroid_tracker.update(rects)

        for (object_id, centroid) in objects.items():
            text = f"ID {object_id}"
            cv2.circle(frame, tuple(centroid), 4, (0, 255, 0), -1)
            cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        for (x1, y1, x2, y2) in rects:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)

        cv2.putText(frame, "Mode: Centroid Tracker", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

    # Deep SORT
    elif tracker_mode == "deepsort":
        detections = []
        for (x1, y1, x2, y2) in rects:
            detections.append(([x1, y1, x2 - x1, y2 - y1], 0.99, 'face'))

        tracks = deep_sort_tracker.update_tracks(detections, frame=frame)

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            center = ((x1 + x2) // 2, (y1 + y2) // 2)

            # Update history
            if track_id not in track_history:
                track_history[track_id] = []
            track_history[track_id].append(center)
            if len(track_history[track_id]) > 20:
                track_history[track_id] = track_history[track_id][-20:]

            # Draw box and path
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
            cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255), 1)

            for i in range(1, len(track_history[track_id])):
                cv2.line(frame, track_history[track_id][i - 1],
                         track_history[track_id][i], (0, 0, 255), 2)

        cv2.putText(frame, "Mode: Deep SORT", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)

    cv2.imshow("Face Tracker", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('1'):
        tracker_mode = "centroid"
    elif key == ord('2'):
        tracker_mode = "deepsort"
    elif key == ord('q') or key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Press '1' for Centroid Tracker, '2' for Deep SORT, 'q' or ESC to quit.
